# Compare original and smoothed topographies for COSMO and ICON

## Python packages

In [4]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path

## Define files:
- COSMO original interpolated from ASTER dataset
- COSMO filtered with operational settings
- ICON original interpolated from ASTER dataset
- ICON ...

In [2]:
f_cosmo_orig = '/store/mch/msopr/swester/vert_coord_files/cosmo_topo/topography_reg_1_nofilter.nc'
f_cosmo_filtered = '/store/mch/msopr/swester/vert_coord_files/cosmo_topo/topography_reg_1_filter.nc'
f_icon_grid = '/store/mch/msopr/swester/grids/alps_R19B08/alps_DOM01.nc'
f_icon_orig = '/store/mch/msopr/swester/vert_coord_files/surf/niter_0.nc'
f_icon_niter1_fac64 = '/store/mch/msopr/swester/vert_coord_files/surf/niter_1_fac_64.nc'

In [6]:
ds_cosmo_orig = xr.open_dataset(f_cosmo_orig)

OSError: [Errno -101] NetCDF: HDF error: b'/store/mch/msopr/swester/vert_coord_files/cosmo_topo/topography_reg_1_nofilter.nc'